# Exploration of NASA Turbofan degredation data set

We import some standard libraries

In [ ]:
import io
import os
import urllib
import zipfile
import json

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


Download and unzip the data set

In [ ]:
data_folder = 'data'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
urllib.request.urlretrieve('https://ti.arc.nasa.gov/m/project/prognostic-repository/CMAPSSData.zip', os.path.join(data_folder, 'CMAPSSData.zip'))

with zipfile.ZipFile(os.path.join(data_folder, 'CMAPSSData.zip'), "r") as zip_ref:
    zip_ref.extractall(data_folder)

We'll focus on the first test data file. 


In [ ]:
raw_data = pd.read_csv('data/train_FD001.txt', delimiter=' ', header=None)
raw_data.head()

We'll drop the last 2 columns as they contain NaNs

In [ ]:
raw_data = raw_data[[f for f in range(0,26)]]
raw_data.head()

We'll add the column headers mentioned in the included readme.txt

In [ ]:
raw_data.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3','s4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14','s15', 's16', 's17', 's18', 's19', 's20', 's21']
raw_data

Let's take a closer look at cycle 1

In [ ]:
raw_data[raw_data['id'] == 1]

We'll next compute the Remaining Useful Life (RUL) variable


In [ ]:
max_cycles = raw_data.groupby(['id'], sort=False)['cycle'].max().reset_index().rename(columns={'cycle':'max_cycle'})
max_cycles.head()

In [ ]:
FD001_df = pd.merge(raw_data, max_cycles, how='inner', on='id')
FD001_df['rul'] = FD001_df['max_cycle'] - FD001_df['cycle']
FD001_df

Lets plot the cyles

In [ ]:
engine = []
for i,r in FD001_df.iterrows():
    rul = r['rul']
    engine.append(rul)
    if rul == 0:
        plt.plot(engine)
        engine = []

plt.grid()